In [23]:
import numpy as np 
import pandas as pd  # type: ignore
import seaborn as sns 
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report, confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.cm as cm
from matplotlib import rcParams
from collections import Counter
from nltk.tokenize import RegexpTokenizer
import re
import string
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding # type: ignore
from tensorflow.keras.models import Model # type: ignore
from tensorflow.keras.optimizers import RMSprop # type: ignore
from tensorflow.keras.preprocessing.text import Tokenizer # type: ignore
from tensorflow.keras.preprocessing import sequence # type: ignore
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [24]:
data = pd.read_csv("/Users/donny/code/training.1600000.processed.noemoticon.csv", encoding = "ISO-8859-1", engine="python")
data.columns = ["label", "time", "date", "query", "username", "text"]
#top 5 records of data
data.head()


,label,time,date,query,username,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [25]:
#five last record of data
data.tail()

,label,time,date,query,username,text
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599998,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


In [26]:
#coloumns/features in data
data.columns


Index(['label', 'time', 'date', 'query', 'username', 'text'], dtype='object')

In [27]:
#lenght of data
print('lenght of data is', len(data))


lenght of data is 1599999


In [28]:
#shape of data
data.shape


(1599999, 6)

In [29]:
#data info
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   label     1599999 non-null  int64 
 1   time      1599999 non-null  int64 
 2   date      1599999 non-null  object
 3   query     1599999 non-null  object
 4   username  1599999 non-null  object
 5   text      1599999 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [30]:
#data types of all coloumns 
data.dtypes

label        int64
time         int64
date        object
query       object
username    object
text        object
dtype: object

In [31]:
#checking null values
np.sum(data.isnull().any(axis=1))

np.int64(0)

In [32]:
#rows and coloumns in the dataset
print('Count of columns in the data is:  ', len(data.columns))


Count of columns in the data is:   6


In [33]:
print('Count of rows in the data is:  ', len(data))

Count of rows in the data is:   1599999


In [34]:
#selecting the text and label coloumn
data=data[['text','label']]
#assigning 1 to positive sentment 4
data['label'][data['label']==4]=1
#sparating postive and negative tweets
data_pos=data[data['label']==1]
data_neg=data[data['label']==0]
#taking one fouth data so we cann run on our machine easily
data_pos=data_pos.iloc[:int(20000)]
data_neg=data_neg.iloc[:int(20000)]
#combing postive and negative tweets
data=pd.concat([data_pos,data_neg])
#making satement text in lower case
data['text']=data['text'].str.lower()
data['text'].tail()


19995                            one more day of holidays 
19996    feeling so down right now .. i hate you damn h...
19997    geez,i hv to read the whole book of personalit...
19998    i threw my sign at donnie and he bent over to ...
19999    @heather2711 good thing i didn't find any then...
Name: text, dtype: object

In [35]:
#cleaning and removing stop words of english
import nltk
from nltk.corpus import stopwords
stopwords_list = stopwords.words('english')
from nltk.corpus import stopwords
",".join(stopwords.words('english'))




"a,about,above,after,again,against,ain,all,am,an,and,any,are,aren,aren't,as,at,be,because,been,before,being,below,between,both,but,by,can,couldn,couldn't,d,did,didn,didn't,do,does,doesn,doesn't,doing,don,don't,down,during,each,few,for,from,further,had,hadn,hadn't,has,hasn,hasn't,have,haven,haven't,having,he,he'd,he'll,her,here,hers,herself,he's,him,himself,his,how,i,i'd,if,i'll,i'm,in,into,is,isn,isn't,it,it'd,it'll,it's,its,itself,i've,just,ll,m,ma,me,mightn,mightn't,more,most,mustn,mustn't,my,myself,needn,needn't,no,nor,not,now,o,of,off,on,once,only,or,other,our,ours,ourselves,out,over,own,re,s,same,shan,shan't,she,she'd,she'll,she's,should,shouldn,shouldn't,should've,so,some,such,t,than,that,that'll,the,their,theirs,them,themselves,then,there,these,they,they'd,they'll,they're,they've,this,those,through,to,too,under,until,up,ve,very,was,wasn,wasn't,we,we'd,we'll,we're,were,weren,weren't,we've,what,when,where,which,while,who,whom,why,will,with,won,won't,wouldn,wouldn't,y,you,you'd,you

In [36]:
import pandas as pd

# Load the dataset properly
data = pd.read_csv(
    '/Users/donny/code/training.1600000.processed.noemoticon.csv',
    encoding='latin-1',
    header=None,
    names=['target', 'ids', 'date', 'flag', 'user', 'text']
)

# Preview the first few rows
print(data.head())
print(data.columns)
STOPWORDS = set(stopwords.words('english'))
def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
data['text'] = data['text'].apply(lambda text: cleaning_stopwords(text))
data['text'].head()


   target         ids                          date      flag  \
0       0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY   
1       0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   
2       0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY   
3       0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
4       0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   

              user                                               text  
0  _TheSpecialOne_  @switchfoot http://twitpic.com/2y1zl - Awww, t...  
1    scotthamilton  is upset that he can't update his Facebook by ...  
2         mattycus  @Kenichan I dived many times for the ball. Man...  
3          ElleCTF    my whole body feels itchy and like its on fire   
4           Karoli  @nationwideclass no, it's not behaving at all....  
Index(['target', 'ids', 'date', 'flag', 'user', 'text'], dtype='object')


0    @switchfoot http://twitpic.com/2y1zl - Awww, t...
1    upset can't update Facebook texting it... migh...
2    @Kenichan I dived many times ball. Managed sav...
3                     whole body feels itchy like fire
4    @nationwideclass no, behaving all. mad. here? ...
Name: text, dtype: object

In [37]:
#Cleaning and removing punctuations¶
english_punctuations = string.punctuation
punctuations_list = english_punctuations
def cleaning_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
data['text']= data['text'].apply(lambda x: cleaning_punctuations(x))
data['text'].tail()

1599995         Just woke up Having school best feeling ever
1599996    TheWDBcom  Very cool hear old Walt interviews ...
1599997                  Are ready MoJo Makeover Ask details
1599998    Happy 38th Birthday boo alll time Tupac Amaru ...
1599999    happy charitytuesday theNSPCC SparksCharity Sp...
Name: text, dtype: object

In [38]:
#cleaning and removing repeating characters
def cleaning_repeating_char(text):
     return re.sub(r'(.)\1+', r'\1', text)
data['text'] = data['text'].apply(lambda x: cleaning_repeating_char(x))
data['text'].tail()


1599995           Just woke up Having schol best feling ever
1599996    TheWDBcom Very col hear old Walt interviews â...
1599997                  Are ready MoJo Makeover Ask details
1599998     Hapy 38th Birthday bo al time Tupac Amaru Shakur
1599999    hapy charitytuesday theNSPC SparksCharity Spea...
Name: text, dtype: object

In [39]:
#cleaning and remving email
def cleaning_email(data):
    return re.sub('@[^\s]+',' ',data)
data['text']=data['text'].apply(lambda x:cleaning_email(x))
data['text'].tail()

1599995           Just woke up Having schol best feling ever
1599996    TheWDBcom Very col hear old Walt interviews â...
1599997                  Are ready MoJo Makeover Ask details
1599998     Hapy 38th Birthday bo al time Tupac Amaru Shakur
1599999    hapy charitytuesday theNSPC SparksCharity Spea...
Name: text, dtype: object

In [40]:
#cleaning and removing URLs
def cleaning_URLs(data):
    return re.sub('((www\.[^\s]+|(https?://[^\s]+)))',' ',data)
data['text'] = data['text'].apply(lambda x: cleaning_URLs(x))
data['text'].tail()

1599995           Just woke up Having schol best feling ever
1599996    TheWDBcom Very col hear old Walt interviews â...
1599997                  Are ready MoJo Makeover Ask details
1599998     Hapy 38th Birthday bo al time Tupac Amaru Shakur
1599999    hapy charitytuesday theNSPC SparksCharity Spea...
Name: text, dtype: object

In [41]:
# cleaning and removing numeric numbers
def cleaning_numbers(data):
    return re.sub('[0-9]+','',data)
data['text'] = data['text'].apply(lambda x: cleaning_numbers(x))
data['text'].tail()


1599995           Just woke up Having schol best feling ever
1599996    TheWDBcom Very col hear old Walt interviews â...
1599997                  Are ready MoJo Makeover Ask details
1599998       Hapy th Birthday bo al time Tupac Amaru Shakur
1599999    hapy charitytuesday theNSPC SparksCharity Spea...
Name: text, dtype: object

In [42]:
#Getting tokenization of tweet text
tokenizer = RegexpTokenizer(r'\w+')
data['text'] = data['text'].apply(lambda x: tokenizer.tokenize(str(x)))
data['text'].head()

0    [switchfot, htptwitpicomyzl, Aw, thats, bumer,...
1    [upset, cant, update, Facebok, texting, it, mi...
2    [Kenichan, I, dived, many, times, bal, Managed...
3               [whole, body, fels, itchy, like, fire]
4    [nationwideclas, no, behaving, al, mad, here, ...
Name: text, dtype: object

In [43]:
# applying steamming 
st=nltk.PorterStemmer()
def stemming_on_text(data):
    text=[st.steam(word)for word in data]
    return data
data['text'].head()

0    [switchfot, htptwitpicomyzl, Aw, thats, bumer,...
1    [upset, cant, update, Facebok, texting, it, mi...
2    [Kenichan, I, dived, many, times, bal, Managed...
3               [whole, body, fels, itchy, like, fire]
4    [nationwideclas, no, behaving, al, mad, here, ...
Name: text, dtype: object

In [44]:
# applying lemmatizer
lm = nltk.WordNetLemmatizer()
def lemmatizer_on_text(data):
    text = [lm.lemmatize(word) for word in data]
    return data

data['text'] = data['text'].apply(lambda x: lemmatizer_on_text(x))
data['text'].head()



0    [switchfot, htptwitpicomyzl, Aw, thats, bumer,...
1    [upset, cant, update, Facebok, texting, it, mi...
2    [Kenichan, I, dived, many, times, bal, Managed...
3               [whole, body, fels, itchy, like, fire]
4    [nationwideclas, no, behaving, al, mad, here, ...
Name: text, dtype: object

In [45]:
data = data.rename(columns={'target': 'label'})
data['label'] = data['label'].replace(4, 1)

X = data['text']
y = data['label']


In [46]:
X = data['text']
y = data['label']
max_len = 500
tok = Tokenizer(num_words=2000)
tok.fit_on_texts(X)
sequences = tok.texts_to_sequences(X)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
sequences_matrix.shape
X_train, X_test, Y_train, Y_test = train_test_split(sequences_matrix, y, test_size=0.3, random_state=2)



In [ ]:
def tensorflow_based_model(): #Defined tensorflow_based_model function for training tenforflow based model
    inputs = Input(name='inputs',shape=[max_len])#step1
    layer = Embedding(2000,50,input_length=max_len)(inputs) #step2
    layer = LSTM(64)(layer) #step3
    layer = Dense(256,name='FC1')(layer) #step4
    layer = Activation('relu')(layer) # step5
    layer = Dropout(0.5)(layer) # step6
    layer = Dense(1,name='out_layer')(layer) #step4 again but this time its giving only one output as because we need to classify the tweet as positive or negative
    layer = Activation('sigmoid')(layer) #step5 but this time activation function is sigmoid for only one output.
    model = Model(inputs=inputs,outputs=layer) #here we are getting the final output value in the model for classification
    return model #function returning the value when we call it
model = tensorflow_based_model() # here we are calling the function of created model
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])  
history=model.fit(X_train,Y_train,batch_size=80,epochs=6, validation_split=0.1)# here we are starting the training of model by feeding the training data
print('Training finished !!')

IndentationError: unexpected indent (782937072.py, line 2)